# 필요 라이브러리 import

In [1]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os
import re
import random
import json
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

%matplotlib inline

# 전처리를 위한 Drive_data_maker class 생성
- all_data_path : 학습을 위한 이미지의 모든 데이터 경로를 한 txt 파일에 write하여 저장하는 함수
- pick_data_path : 사용할 이미지의 경로만 한 txt파일에 write하여 저장하는 함수
- shuffle_pick : 학습시 데이터를 골고루 사용할 수 있도록 shuffle하는 함수
- resize_img : image resizing 함수
- data_split_txt : train / valid / test로 split하는 함수
- json_path : 사용할 이미지의 json파일 경로만 모아주는 함수
- json_to_txt : json파일을 불러와 모델 학습을 위해 (label, x, y, w, h)로 변환하여 txt파일에 저장 시키는 함수


In [2]:
class Drive_data_maker():
    def __init__(self, data_path, make_path):
        self.data_path = data_path
        self.make_path = make_path

    def all_data_path(self, name='train'):
        print(f'All data path save file name: {name}_img_path.txt')
        f = open(self.make_path + f'{name}_img_path.txt', 'w')
        for folder in tqdm(os.listdir(self.data_path)):
            data_sub_path = self.data_path + '/' + folder
            for img in os.listdir(data_sub_path):
                if img[-3:] == 'jpg':
                    f.write(data_sub_path + '/' + img + '\n')
        f.close()
    
    def pick_data_path(self, input_txt, name, label):
        print(f'Pick data path save file name: {name}_img_path.txt')

        f = open(self.make_path + input_txt, 'r')
        t = open(self.make_path + f'{name}_img_path.txt', 'w')

        all_list = f.readlines()
        for line in all_list:
            if label in line:
                t.write(line)
            else:
                pass

        f.close()
        t.close()
        

    def shuffle_pick(self, txt_data, pick_num = -1):
        f = open(self.make_path + txt_data, 'r')
        pick_img_path = f.readlines()
        f.close()

        random.shuffle(pick_img_path)

        pick_img_path = pick_img_path[:pick_num]

        return pick_img_path
    

    def resize_img(self, data):
        for path in tqdm(data):

            imgs = cv2.imread(path)
            imgs = cv2.resize(imgs,(416, 416))

            gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
            imgs = np.zeros_like(imgs)

            imgs[:,:,0] = gray
            imgs[:,:,1] = gray
            imgs[:,:,2] = gray

            name = path.split('/')[-1].strip()

            save_path = self.make_path + name
            cv2.imwrite(save_path, imgs)

    def data_split_txt(self, data1, data2, split_list=[6, 2, 2]):        
        f = open(self.make_path + 'mix_data.txt', 'w')
        for a, b in zip(data1, data2):
            f.write(a)
            f.write(b)

        f.close()

        print('Maked mix_data.txt')

        t = open(self.make_path + 'mix_data.txt', 'r')
        mix_data = t.readlines()
        t.close()

        len_all = len(mix_data)
        a = int(len_all * (split_list[0] / sum(split_list)))
        b = int(a + (len_all * (split_list[1] / sum(split_list))))

        train_data = mix_data[:a]
        valid_data = mix_data[a:b]
        test_data = mix_data[b:]

        print('Maked [train, vaild, test] txt files')
        
        data_list = [train_data , valid_data, test_data]
        make_list = ['train', 'valid', 'test']

        for datas, maker in zip(data_list, make_list):
            f = open(self.make_path + f'{maker}.txt', 'w')
            for data in datas:
                f.write(data)
            
            f.close()


    def json_path(self):
        f = open(self.make_path + 'mix_data.txt', 'r')
        t = open(self.make_path + 'json_path.txt', 'w')
        
        read_path = f.readlines()
        for path in read_path:
            read_json = path[:-4] + 'json' + '\n'
            t.write(read_json)

        f.close()
        t.close()


    def json_to_txt(objeck_list):
        f = open(make_path + 'json_path.txt', 'r')
        json_path = f.readlines()

        for i in tqdm(range(len(json_path))):
            bass_json = json_path[i].strip()
            j = open(bass_json, 'r')
            read_json = json.load(j)

            json_path[i].split('/')
            t = open(make_path + 'json/' + json_path[i].split('/')[-1][:-5] + 'txt', 'w')

            object_list = objeck_list
            for obj in object_list:
                opened = read_json['ObjectInfo']['BoundingBox'][obj]['Opened']
                position = read_json['ObjectInfo']['BoundingBox'][obj]['Position']

                if opened: eye_label = 1
                else: eye_label = 0

                x1, y1, x2, y2 = position[0], position[1], position[2], position[3]
                
                x = (x1 + x2) / (2 * 800)
                y = (y1 + y2) / (2 * 1280)
                w = (x2 - x1) / 800
                h = (y2 - y1) / 1280
                
                t.write(f'{eye_label} {x:f} {y:f} {w:f} {h:f}\n')
        
        f.close()
        t.close()
        j.close()

# test 실행

## google drive import 및 필요 path 입력

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
data_path = '/content/drive/MyDrive/Colab Notebooks/alpaco/팀플 1조 (객체인식) /data/training'
make_path = '/content/drive/MyDrive/Colab Notebooks/3rd/data/'

drive_data = Drive_data_maker(data_path, make_path)

## 실행

In [13]:
drive_data.all_data_path()

All data path save file name: train_img_path.txt


FileNotFoundError: ignored

In [6]:
drive_data.pick_data_path('train_img_path.txt', 'normal', '정상주시')
drive_data.pick_data_path('train_img_path.txt', 'sleep', '졸음재현')

Pick data path save file name: normal_img_path.txt


FileNotFoundError: ignored

In [7]:
normal_img_path = drive_data.shuffle_pick('normal_img_path.txt', pick_num=5000)
sleep_img_path = drive_data.shuffle_pick('sleep_img_path.txt', pick_num=5000)

FileNotFoundError: ignored

In [8]:
drive_data.resize_img()

TypeError: ignored

In [9]:
data_list = ['train', 'valid', 'test']

for data in data_list:
    f = open(make_path + f'{data}.txt', 'r')
    read_data = f.readlines()

    print(len(read_data))

FileNotFoundError: ignored

In [ ]:
drive_data.json_path()

In [ ]:
drive_data.json_to_txt()

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
drive_data.data_split_txt(normal_img_path, sleep_img_path)

Maked mix_data.txt
Maked [train, vaild, test] txt files
